In [26]:
from sklearn.preprocessing import LabelEncoder, StandardScaler,OrdinalEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

In [32]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample  = pd.read_csv('sample_submission.csv')
train

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15674932,Okwudilichukwu,668,France,Male,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,1,15749177,Okwudiliolisa,627,France,Male,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,2,15694510,Hsueh,678,France,Male,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,3,15741417,Kao,581,France,Male,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,4,15766172,Chiemenam,716,Spain,Male,33.0,5,0.00,2,1.0,1.0,15068.83,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165029,165029,15667085,Meng,667,Spain,Female,33.0,2,0.00,1,1.0,1.0,131834.75,0
165030,165030,15665521,Okechukwu,792,France,Male,35.0,3,0.00,1,0.0,0.0,131834.45,0
165031,165031,15664752,Hsia,565,France,Male,31.0,5,0.00,1,1.0,1.0,127429.56,0
165032,165032,15689614,Hsiung,554,Spain,Female,30.0,7,161533.00,1,0.0,1.0,71173.03,0


In [30]:
#transforming categorical columns to encoded values
categorical_columns = ['Surname','Geography','Gender']
en = OrdinalEncoder()
en.fit(train[categorical_columns])
transform = en.transform(train[categorical_columns])
train[categorical_columns] = transform

In [31]:

train

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15674932,1924.0,668,0.0,1.0,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,1,15749177,1925.0,627,0.0,1.0,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,2,15694510,1178.0,678,0.0,1.0,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,3,15741417,1299.0,581,0.0,1.0,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,4,15766172,467.0,716,2.0,1.0,33.0,5,0.00,2,1.0,1.0,15068.83,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165029,165029,15667085,1701.0,667,2.0,0.0,33.0,2,0.00,1,1.0,1.0,131834.75,0
165030,165030,15665521,1918.0,792,0.0,1.0,35.0,3,0.00,1,0.0,0.0,131834.45,0
165031,165031,15664752,1172.0,565,0.0,1.0,31.0,5,0.00,1,1.0,1.0,127429.56,0
165032,165032,15689614,1176.0,554,2.0,0.0,30.0,7,161533.00,1,0.0,1.0,71173.03,0


array(['Abazu', 'Abbie', 'Abbott', ..., 'Zuev', 'Zuyev', 'Zuyeva'],
      dtype=object)